In [2]:
import json
import os

# sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
## open all tweets in folder of raw tweets

tweets = os.listdir("raw_tweets_entitites_all_tweets_1000")
tweets = [i for i in tweets if i.endswith("json")]

In [3]:
### merge all tweets into one big list

big = []
for i in tweets:
    with open("all_tweets_1000/"+i, "r") as f:
        t = json.load(f)
    big.extend(t)

In [4]:
len(big)

1013650

In [5]:
### remove empty tweets

t = [i for i in big if str(i["entity_name"])!="nan"]
len(t)

990338

In [6]:
### import other modules

import os
import json
from pdb import set_trace
import pandas as pd
import ast
import yaml
import matplotlib.pyplot as plt
import csv
import numpy as np
import math
from scipy import stats
from collections import Counter
import string
import re
from cleantext import clean

# sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from nltk.stem.snowball import SnowballStemmer
snow_stemmer = SnowballStemmer(language='english')

#stopwords
from nltk.corpus import stopwords

#lemmatizer
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
  
lemmatizer = WordNetLemmatizer()

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zhuang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
### different stemmers and lemmatizers to experiment with

def lemmetize_print(words):
    a = []
    tokens = word_tokenize(words)
    for token in tokens:
        lemmetized_word = lemmatizer.lemmatize(token)
        a.append(lemmetized_word)
    return (" ".join(a))

lemmetize_print("cats cats")



def stem_print(words):
    a = []
    tokens = word_tokenize(words)
    for token in tokens:
        stemmed_word = snow_stemmer.stem(token)
        a.append(stemmed_word)
        
    return (" ".join(a))

lemmetize_print("cats cats")
stem_print("universal universe")

'univers univers'

In [8]:
### make list of stop words, add 'rt' to list

s = stopwords.words('english')
s.append("rt")

## remove stop words from stopwords dictionary that are negative or positive (we only want to remove neutral words for sentiment analysis purposes)
s = [i for i in s if analyzer.polarity_scores(i)['compound']==0]

In [10]:
### remove stopwords function
def remove_stopwords(sen):

    word_tokens = word_tokenize(sen)

    filtered_sentence = [w for w in word_tokens if not w.lower() in s]

    " ".join(filtered_sentence)
    
    return " ".join(filtered_sentence)

### make dataframe of all tweets

df = pd.DataFrame(t)





In [11]:
## begin processing text

## lowercase all

df['text'] = df['text'].map(lambda x: str(x).lower())

In [12]:
### remove urls and handles

df['text'] = df['text'].map(lambda x: re.sub("@[^ ]+", "", x, count=0, flags=0))
df['text'] = df['text'].map(lambda x: re.sub("http[^ ]*", "", x, count=0, flags=0))

In [13]:
### remove stopwords, add lemmatized and stemmed version

df['text_nostop'] = df['text'].map(lambda x: remove_stopwords(x))
df['text_nostop_lemma'] = df['text_nostop'].map(lambda x: lemmetize_print(x))
df['text_nostop_stem'] = df['text_nostop'].map(lambda x: stem_print(x))

In [14]:
### sentiment analysis of lemmatized and stemmed cleaned tweets


df['sentiment_nostop'] = df['text_nostop'].map(lambda x: analyzer.polarity_scores(x)["compound"])
df['sentiment_nostop_lemma'] = df['text_nostop_lemma'].map(lambda x: analyzer.polarity_scores(x)["compound"])
df['sentiment_nostop_stem'] = df['text_nostop_stem'].map(lambda x: analyzer.polarity_scores(x)["compound"])
    
    

In [16]:
### write to json

df.to_csv("full_dataframe_with_sentiment.csv")

In [34]:
### write all merged tweets to json in case we need it later

with open("all_entity_tweets_1000_each.json", "w") as outfile:
    json_object = json.dumps(t, indent=4)
    outfile.write(json_object)

In [7]:
##  we also tried removing emojis but it made no difference

import re
from cleantext import clean

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [3]:
import pandas as pd
df = pd.read_csv("full_dataframe_with_sentiment.csv")

C:\Users\zhuang\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
## remove emojis from text 

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

df['text_nostop_lemma_emoji'] = df['text_nostop_lemma'].map(lambda x: clean(x, no_emoji=True))
df['text_nostop_stem_emoji'] = df['text_nostop_stem'].map(lambda x: clean(x, no_emoji=True))

df.to_csv("full_dataframe_with_sentiment.csv")